# Tutorial: CS Amplifier Design with gm/ID Methodology
## IHP SG13G2 130nm PDK

---

### Tutorial Objectives

After completing this tutorial, you will be able to:

1. **Understand** the gm/ID methodology for analog circuit design
2. **Use** pre-characterized lookup tables for transistor sizing
3. **Analyze** trade-offs between speed, gain, and power consumption
4. **Verify** designs using ngspice simulation
5. **Export** parameters for use in xschem

### Prerequisites

- Basic MOSFET knowledge (operating regions)
- Basic Python
- Familiarity with xschem (optional, for final section)

---
## Part 1: Introduction to the gm/ID Methodology

### The Problem: How to Size a Transistor?

When designing an analog circuit, we need to choose:
- **W** (transistor width)
- **L** (channel length)
- The **operating point** (VGS, VDS)

#### Classical Approach (Square Law Equation)

The quadratic MOSFET equation in saturation:

$$I_D = \frac{1}{2} \mu C_{ox} \frac{W}{L} (V_{GS} - V_{th})^2$$

**Problems:**
- Only valid in strong inversion
- Ignores short-channel effects
- Doesn't work well in moderate/weak inversion

#### The gm/ID Approach

Instead of using the square law equation, we use **real simulation data** organized in lookup tables.

The key parameter is **gm/ID** (transconductance efficiency):

$$\frac{g_m}{I_D} = \frac{\partial I_D / \partial V_{GS}}{I_D}$$

This parameter is a "design knob" that controls the operating region:

| gm/ID (S/A) | Region | Characteristics |
|-------------|--------|----------------|
| 5-10 | Strong Inversion | High speed, high power |
| 10-15 | Moderate Inversion | Optimal balance |
| 15-25 | Weak Inversion | Low power, lower speed |

---
## Part 2: Loading the Lookup Tables

The lookup tables contain transistor parameters simulated for multiple combinations of VGS, VDS, VBS, and L.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Widgets for interactivity
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown

# Mosplot libraries
from mosplot.plot import load_lookup_table, Mosfet, Expression
from mosplot.plot.util import evaluate_expression

# Plot configuration
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

# Color-blind friendly palette
TOL_BRIGHT = ['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', '#BBBBBB']
MARKERS = ['o', 's', '^', 'D', 'v', 'p', '*']

print("Libraries loaded successfully.")

In [ ]:
# Load the NMOS lookup table
DATA_DIR = Path("../ihp_sg13g2_data")
nmos_path = DATA_DIR / "sg13_lv_nmos.npz"

if not nmos_path.exists():
    print(f"ERROR: File not found {nmos_path}")
    print("Make sure you have generated the tables with ihp_lookup_generator.py")
else:
    lt = load_lookup_table(str(nmos_path))
    print(f"Table loaded: {nmos_path}")
    print(f"Available models: {list(lt.keys())}")

In [ ]:
# Explore the table contents
nmos_data = lt['sg13_lv_nmos']

print("=== Lookup Table Contents ===")
print(f"\nAvailable parameters:")
for key in nmos_data.keys():
    if isinstance(nmos_data[key], np.ndarray):
        print(f"  {key}: shape = {nmos_data[key].shape}")
    else:
        print(f"  {key}: {nmos_data[key]}")

print(f"\nChannel lengths (L):")
print(f"  {[f'{L*1e6:.2f} µm' for L in nmos_data['length']]}")

print(f"\nVGS range: {nmos_data['vgs'].min():.2f}V to {nmos_data['vgs'].max():.2f}V")
print(f"VDS range: {nmos_data['vds'].min():.2f}V to {nmos_data['vds'].max():.2f}V")
print(f"VBS values: {nmos_data['vbs']}V")
print(f"\nWidth used in simulation (W): {nmos_data['width']*1e6:.1f} µm")

---
## Part 3: The 4 Fundamental Charts

The gm/ID methodology uses 4 key charts for design. Let's look at each one and what it tells us.

In [ ]:
# Create Mosfet object for analysis
# We fix VDS = 0.6V (saturation) and VBS = 0V
nmos = Mosfet(
    lookup_table=lt,
    mos="sg13_lv_nmos",
    vbs=0.0,
    vds=0.6,
    vgs=(0.3, 1.2)  # VGS range to analyze
)

print(f"Mosfet configured:")
print(f"  VDS = 0.6V (fixed)")
print(f"  VBS = 0.0V (fixed)")
print(f"  VGS = 0.3V to 1.2V (sweep)")
print(f"  Lengths: {[f'{L*1e6:.2f}µm' for L in nmos.length]}")

In [ ]:
# Extract data for the charts
gmid_data, _ = evaluate_expression(nmos.gmid_expression, nmos.extracted_table)
gain_data, _ = evaluate_expression(nmos.gain_expression, nmos.extracted_table)
ft_data, _ = evaluate_expression(nmos.transist_frequency_expression, nmos.extracted_table)
id_w_data, _ = evaluate_expression(nmos.current_density_expression, nmos.extracted_table)

print(f"Data extracted: {gmid_data.shape[0]} lengths × {gmid_data.shape[1]} VGS points")

### Chart 1: Transit Frequency (fT) vs gm/ID

**What it shows:** The maximum speed of the transistor.

**Key observation:** fT decreases as gm/ID increases. This means operating in weak inversion (high gm/ID) reduces speed.

In [ ]:
fig, ax = plt.subplots()

for l_idx, L in enumerate(nmos.length):
    valid = np.isfinite(gmid_data[l_idx, :]) & (ft_data[l_idx, :] > 0)
    ax.semilogy(
        gmid_data[l_idx, valid], ft_data[l_idx, valid] / 1e9,
        color=TOL_BRIGHT[l_idx % len(TOL_BRIGHT)],
        marker=MARKERS[l_idx % len(MARKERS)],
        markevery=10,
        label=f'L={L*1e6:.2f}µm'
    )

ax.set_xlabel('gm/ID (S/A)')
ax.set_ylabel('fT (GHz)')
ax.set_title('Transit Frequency vs gm/ID')
ax.set_xlim(5, 25)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Notice: Shorter lengths have higher fT (they are faster)")

### Chart 2: Intrinsic Gain (gm/gds) vs gm/ID

**What it shows:** The maximum gain the transistor can provide (without load).

**Key observation:** Gain increases with L and with gm/ID.

In [ ]:
fig, ax = plt.subplots()

for l_idx, L in enumerate(nmos.length):
    valid = np.isfinite(gmid_data[l_idx, :]) & (gain_data[l_idx, :] > 0)
    ax.semilogy(
        gmid_data[l_idx, valid], gain_data[l_idx, valid],
        color=TOL_BRIGHT[l_idx % len(TOL_BRIGHT)],
        marker=MARKERS[l_idx % len(MARKERS)],
        markevery=10,
        label=f'L={L*1e6:.2f}µm'
    )

# Reference line: Gain = 10 (20 dB)
ax.axhline(y=10, color='gray', linestyle='--', alpha=0.7, label='Av = 10 (20 dB)')

ax.set_xlabel('gm/ID (S/A)')
ax.set_ylabel('gm/gds (V/V)')
ax.set_title('Intrinsic Gain vs gm/ID')
ax.set_xlim(5, 25)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Notice: Longer lengths have higher gain (lower gds)")

### Chart 3: Current Density (ID/W) vs gm/ID

**What it shows:** How much current flows per unit width.

**Main use:** To calculate W once we know ID.

In [ ]:
fig, ax = plt.subplots()

for l_idx, L in enumerate(nmos.length):
    valid = np.isfinite(gmid_data[l_idx, :]) & (id_w_data[l_idx, :] > 0)
    ax.semilogy(
        gmid_data[l_idx, valid], id_w_data[l_idx, valid],
        color=TOL_BRIGHT[l_idx % len(TOL_BRIGHT)],
        marker=MARKERS[l_idx % len(MARKERS)],
        markevery=10,
        label=f'L={L*1e6:.2f}µm'
    )

ax.set_xlabel('gm/ID (S/A)')
ax.set_ylabel('ID/W (A/m)')
ax.set_title('Current Density vs gm/ID')
ax.set_xlim(5, 25)
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 This chart lets you calculate: W = ID / (ID/W)")

---
## Part 4: Interactive Step-by-Step Design

Now let's design a common source amplifier. Use the interactive controls to experiment with different values.

In [ ]:
# === DESIGN SPECIFICATIONS ===
# Modify these values to experiment

VDD = 1.2           # Supply voltage (V)
Av_target = 10.0    # Target gain (V/V) - 20 dB
BW_target = 100e6   # Target bandwidth (Hz) - 100 MHz
CL = 100e-15        # Load capacitance (F) - 100 fF
VDS_op = 0.6        # Operating VDS (V)

print("=== Design Specifications ===")
print(f"  VDD = {VDD} V")
print(f"  Target gain (Av) = {Av_target} V/V ({20*np.log10(Av_target):.1f} dB)")
print(f"  Bandwidth (BW) = {BW_target/1e6:.0f} MHz")
print(f"  Load capacitance (CL) = {CL*1e15:.0f} fF")

### Step 1: Calculate Required gm

For an amplifier with capacitive load, the bandwidth is given by:

$$BW = \frac{g_m}{2\pi C_L}$$

Solving for gm:

$$g_m = 2\pi \cdot BW \cdot C_L$$

In [ ]:
# STEP 1: Calculate required gm
gm_required = 2 * np.pi * BW_target * CL

print(f"=== STEP 1: Required gm ===")
print(f"  gm = 2π × {BW_target/1e6:.0f} MHz × {CL*1e15:.0f} fF")
print(f"  gm = {gm_required*1e6:.2f} µS")

### Step 2: Select Channel Length (L)

Use the dropdown to select L and observe if it meets the gain requirement.

In [ ]:
# Widget to select L
L_options = {f'{L*1e6:.2f} µm': L for L in nmos.length}

L_dropdown = widgets.Dropdown(
    options=L_options,
    value=nmos.length[2],  # Initial value: 0.18 µm
    description='Length L:',
    style={'description_width': 'initial'}
)

output_L = widgets.Output()

def update_L_plot(change):
    with output_L:
        output_L.clear_output(wait=True)
        L_selected = change['new']
        L_idx = np.argmin(np.abs(np.array(nmos.length) - L_selected))
        
        fig, ax = plt.subplots(figsize=(10, 5))
        
        # Plot all lengths in gray
        for l_idx, L in enumerate(nmos.length):
            valid = np.isfinite(gmid_data[l_idx, :]) & (gain_data[l_idx, :] > 0)
            ax.semilogy(gmid_data[l_idx, valid], gain_data[l_idx, valid],
                       color='lightgray', alpha=0.5)
        
        # Highlight selected length
        valid = np.isfinite(gmid_data[L_idx, :]) & (gain_data[L_idx, :] > 0)
        ax.semilogy(gmid_data[L_idx, valid], gain_data[L_idx, valid],
                   color=TOL_BRIGHT[0], linewidth=3,
                   label=f'L = {L_selected*1e6:.2f} µm (SELECTED)')
        
        # Target gain line
        ax.axhline(y=Av_target, color='red', linestyle='--', 
                   label=f'Target: Av = {Av_target} V/V')
        
        ax.set_xlabel('gm/ID (S/A)')
        ax.set_ylabel('Gain (V/V)')
        ax.set_title('L Selection to Meet Gain Requirement')
        ax.set_xlim(5, 25)
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Check if it meets requirement
        max_gain = np.max(gain_data[L_idx, valid])
        gain_at_10 = gain_data[L_idx, np.argmin(np.abs(gmid_data[L_idx, :] - 10))]
        
        if gain_at_10 >= Av_target:
            print(f"✅ L = {L_selected*1e6:.2f} µm MEETS the requirement")
        else:
            print(f"❌ L = {L_selected*1e6:.2f} µm DOES NOT MEET - consider larger L")
        print(f"   Gain @ gm/ID=10: {gain_at_10:.1f} V/V ({20*np.log10(gain_at_10):.1f} dB)")
        print(f"   Maximum gain: {max_gain:.1f} V/V ({20*np.log10(max_gain):.1f} dB)")

L_dropdown.observe(update_L_plot, names='value')
display(L_dropdown)
display(output_L)

# Initial trigger
update_L_plot({'new': L_dropdown.value})

### Step 3: Select Operating gm/ID

The gm/ID value controls:
- **Low gm/ID (5-10)**: Strong inversion, high speed, high power
- **High gm/ID (15-20)**: Weak inversion, low speed, low power

In [ ]:
# Widgets for interactive design
gmid_slider = widgets.FloatSlider(
    value=10.0,
    min=5.0,
    max=20.0,
    step=0.5,
    description='gm/ID (S/A):',
    continuous_update=True,
    style={'description_width': 'initial'}
)

output_design = widgets.Output()

def update_design(change):
    with output_design:
        output_design.clear_output(wait=True)
        
        gmid_target = gmid_slider.value
        L_selected = L_dropdown.value
        L_idx = np.argmin(np.abs(np.array(nmos.length) - L_selected))
        
        # Find point closest to target gm/ID
        valid = np.isfinite(gmid_data[L_idx, :]) & (gmid_data[L_idx, :] > 0)
        valid_indices = np.where(valid)[0]
        closest_idx = valid_indices[np.argmin(np.abs(gmid_data[L_idx, valid] - gmid_target))]
        
        # Extract values at operating point
        gmid_op = gmid_data[L_idx, closest_idx]
        gain_op = gain_data[L_idx, closest_idx]
        ft_op = ft_data[L_idx, closest_idx]
        id_w_op = id_w_data[L_idx, closest_idx]
        vgs_op = nmos.vgs[closest_idx]
        
        # Calculate dimensions
        id_required = gm_required / gmid_op
        w_required = id_required / id_w_op
        
        # Create visualization
        fig, axes = plt.subplots(1, 3, figsize=(14, 4))
        
        # Plot 1: Gain
        ax = axes[0]
        valid_plot = np.isfinite(gmid_data[L_idx, :]) & (gain_data[L_idx, :] > 0)
        ax.semilogy(gmid_data[L_idx, valid_plot], gain_data[L_idx, valid_plot], 
                   color=TOL_BRIGHT[0], linewidth=2)
        ax.plot(gmid_op, gain_op, 'r*', markersize=15, label='Operating point')
        ax.axhline(y=Av_target, color='gray', linestyle='--', alpha=0.7)
        ax.set_xlabel('gm/ID (S/A)')
        ax.set_ylabel('Gain (V/V)')
        ax.set_title('Gain')
        ax.set_xlim(5, 20)
        ax.grid(True, alpha=0.3)
        
        # Plot 2: fT
        ax = axes[1]
        valid_plot = np.isfinite(gmid_data[L_idx, :]) & (ft_data[L_idx, :] > 0)
        ax.semilogy(gmid_data[L_idx, valid_plot], ft_data[L_idx, valid_plot]/1e9, 
                   color=TOL_BRIGHT[1], linewidth=2)
        ax.plot(gmid_op, ft_op/1e9, 'r*', markersize=15)
        ax.axhline(y=BW_target/1e9, color='gray', linestyle='--', alpha=0.7)
        ax.set_xlabel('gm/ID (S/A)')
        ax.set_ylabel('fT (GHz)')
        ax.set_title('Speed (fT)')
        ax.set_xlim(5, 20)
        ax.grid(True, alpha=0.3)
        
        # Plot 3: ID/W
        ax = axes[2]
        valid_plot = np.isfinite(gmid_data[L_idx, :]) & (id_w_data[L_idx, :] > 0)
        ax.semilogy(gmid_data[L_idx, valid_plot], id_w_data[L_idx, valid_plot], 
                   color=TOL_BRIGHT[2], linewidth=2)
        ax.plot(gmid_op, id_w_op, 'r*', markersize=15)
        ax.set_xlabel('gm/ID (S/A)')
        ax.set_ylabel('ID/W (A/m)')
        ax.set_title('Current Density')
        ax.set_xlim(5, 20)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Show results
        print("=" * 50)
        print("DESIGN RESULTS")
        print("=" * 50)
        print(f"\n📐 SELECTED PARAMETERS:")
        print(f"   L = {L_selected*1e6:.3f} µm")
        print(f"   gm/ID = {gmid_op:.2f} S/A")
        print(f"   VGS = {vgs_op:.3f} V")
        
        print(f"\n📊 CALCULATED DIMENSIONS:")
        print(f"   W = {w_required*1e6:.3f} µm")
        print(f"   ID = {id_required*1e6:.3f} µA")
        print(f"   gm = {gm_required*1e6:.3f} µS")
        
        print(f"\n🎯 SPECIFICATION VERIFICATION:")
        gain_check = "✅" if gain_op >= Av_target else "❌"
        ft_check = "✅" if ft_op >= BW_target else "❌"
        print(f"   {gain_check} Gain: {gain_op:.1f} V/V ({20*np.log10(gain_op):.1f} dB) [target: {Av_target} V/V]")
        print(f"   {ft_check} fT: {ft_op/1e9:.1f} GHz [margin over BW: {ft_op/BW_target:.0f}x]")
        print(f"   Power: {VDD * id_required * 1e6:.2f} µW")

gmid_slider.observe(update_design, names='value')
L_dropdown.observe(update_design, names='value')

display(widgets.VBox([L_dropdown, gmid_slider]))
display(output_design)

# Initial trigger
update_design({'new': gmid_slider.value})

---
## Part 5: Verification with Ngspice

Let's verify the design by simulating with ngspice.

In [ ]:
# Function to generate netlist
def generate_netlist(W, L, VGS, VDS=0.6):
    """Generates an ngspice netlist for operating point verification."""
    
    pdk_root = os.environ.get('PDK_ROOT', '/path/to/IHP-Open-PDK')
    lib_path = f"{pdk_root}/ihp-sg13g2/libs.tech/ngspice/models/cornerMOSlv.lib"
    
    netlist = f"""* CS Amplifier - Operating Point Verification
* Generated from gm/ID Tutorial

.param W_DESIGN = {W:.6e}
.param L_DESIGN = {L:.6e}
.param VGS_OP = {VGS:.4f}
.param VDS_OP = {VDS:.2f}

.lib '{lib_path}' mos_tt

VGS gate 0 DC={{VGS_OP}}
VDS drain 0 DC={{VDS_OP}}

X1 drain gate 0 0 sg13_lv_nmos L={{L_DESIGN}} W={{W_DESIGN}} ng=1 m=1

.op

.control
op
print @n.x1.nsg13_lv_nmos[id]
print @n.x1.nsg13_lv_nmos[gm]
print @n.x1.nsg13_lv_nmos[gds]
print @n.x1.nsg13_lv_nmos[vth]
.endc

.end
"""
    return netlist

# Get current design values
L_selected = L_dropdown.value
L_idx = np.argmin(np.abs(np.array(nmos.length) - L_selected))
gmid_target = gmid_slider.value

valid = np.isfinite(gmid_data[L_idx, :]) & (gmid_data[L_idx, :] > 0)
valid_indices = np.where(valid)[0]
closest_idx = valid_indices[np.argmin(np.abs(gmid_data[L_idx, valid] - gmid_target))]

gmid_op = gmid_data[L_idx, closest_idx]
id_w_op = id_w_data[L_idx, closest_idx]
vgs_op = nmos.vgs[closest_idx]
id_required = gm_required / gmid_op
w_required = id_required / id_w_op

# Generate and show netlist
netlist = generate_netlist(w_required, L_selected, vgs_op)
print("=== GENERATED NETLIST ===")
print(netlist)

In [ ]:
# Save and run simulation
import subprocess
import tempfile
import re

def run_ngspice_op(netlist):
    """Run ngspice and extract .op results"""
    
    with tempfile.NamedTemporaryFile(mode='w', suffix='.cir', delete=False) as f:
        f.write(netlist)
        netlist_path = f.name
    
    try:
        result = subprocess.run(
            ['ngspice', '-b', netlist_path],
            capture_output=True,
            text=True,
            timeout=30
        )
        output = result.stdout + result.stderr
        
        # Parse results
        results = {}
        patterns = {
            'id': r'\[id\]\s*=\s*([\d.e+-]+)',
            'gm': r'\[gm\]\s*=\s*([\d.e+-]+)',
            'gds': r'\[gds\]\s*=\s*([\d.e+-]+)',
            'vth': r'\[vth\]\s*=\s*([\d.e+-]+)'
        }
        
        for param, pattern in patterns.items():
            match = re.search(pattern, output)
            if match:
                results[param] = float(match.group(1))
        
        return results, output
        
    except subprocess.TimeoutExpired:
        return None, "Timeout: Simulation took too long"
    except FileNotFoundError:
        return None, "Error: ngspice not found. Make sure it is installed."
    finally:
        os.unlink(netlist_path)

print("Running ngspice simulation...")
sim_results, sim_output = run_ngspice_op(netlist)

if sim_results:
    print("\n=== SIMULATION RESULTS ===")
    print(f"  ID  = {sim_results.get('id', 0)*1e6:.3f} µA")
    print(f"  gm  = {sim_results.get('gm', 0)*1e6:.3f} µS")
    print(f"  gds = {sim_results.get('gds', 0)*1e6:.3f} µS")
    print(f"  Vth = {sim_results.get('vth', 0):.3f} V")
    
    if 'gm' in sim_results and 'id' in sim_results:
        gmid_sim = sim_results['gm'] / sim_results['id']
        print(f"  gm/ID = {gmid_sim:.2f} S/A")
    
    if 'gm' in sim_results and 'gds' in sim_results:
        gain_sim = sim_results['gm'] / sim_results['gds']
        print(f"  Gain = {gain_sim:.1f} V/V ({20*np.log10(gain_sim):.1f} dB)")
else:
    print(f"\n⚠️  Could not run simulation:")
    print(sim_output[:500])

---
## Part 6: Experimentation Exercises

Now it's your turn to experiment. Modify the parameters and observe the effects.

### Exercise 1: Increase Bandwidth to 200 MHz

**Question:** If you need BW = 200 MHz (double), what changes in the design?

**Instructions:**
1. Modify `BW_target = 200e6` in the specifications cell
2. Re-run the design cells
3. Observe how W and ID change

In [ ]:
# Your code here - Experiment with BW = 200 MHz
BW_exercise = 200e6  # Modify this value
gm_exercise = 2 * np.pi * BW_exercise * CL

print(f"With BW = {BW_exercise/1e6:.0f} MHz:")
print(f"  Required gm = {gm_exercise*1e6:.2f} µS (before: {gm_required*1e6:.2f} µS)")
print(f"  Increase: {gm_exercise/gm_required:.1f}x")

### Exercise 2: Achieve Higher Gain (30 dB)

**Question:** If you need Av = 30 dB (~32 V/V), what length L should you use?

**Instructions:**
1. Look at the Gain vs gm/ID chart
2. Find which L provides gain > 32 V/V
3. Consider the impact on fT

In [ ]:
# Analysis for exercise 2
Av_target_30dB = 10**(30/20)  # ~31.6 V/V

print(f"Target gain: {Av_target_30dB:.1f} V/V (30 dB)\n")
print("Maximum gain by length (@ gm/ID=10):")
print("-" * 40)

for l_idx, L in enumerate(nmos.length):
    valid = np.isfinite(gmid_data[l_idx, :]) & (gain_data[l_idx, :] > 0)
    idx_10 = np.argmin(np.abs(gmid_data[l_idx, valid] - 10))
    gain_at_10 = gain_data[l_idx, valid][idx_10]
    ft_at_10 = ft_data[l_idx, valid][idx_10]
    
    check = "✅" if gain_at_10 >= Av_target_30dB else "  "
    print(f"{check} L = {L*1e6:.2f} µm: Av = {gain_at_10:.1f} V/V, fT = {ft_at_10/1e9:.1f} GHz")

### Exercise 3: Speed vs Power Trade-off

Compare two designs:
- **Design A:** gm/ID = 8 (strong inversion, fast)
- **Design B:** gm/ID = 15 (moderate/weak inversion, low power)

In [ ]:
# Comparison of two operating points
L_fixed = 0.18e-6  # Use L = 0.18 µm for both
L_idx = np.argmin(np.abs(np.array(nmos.length) - L_fixed))

def get_design_point(gmid_target):
    valid = np.isfinite(gmid_data[L_idx, :]) & (gmid_data[L_idx, :] > 0)
    valid_indices = np.where(valid)[0]
    closest_idx = valid_indices[np.argmin(np.abs(gmid_data[L_idx, valid] - gmid_target))]
    
    gmid_op = gmid_data[L_idx, closest_idx]
    gain_op = gain_data[L_idx, closest_idx]
    ft_op = ft_data[L_idx, closest_idx]
    id_w_op = id_w_data[L_idx, closest_idx]
    
    id_req = gm_required / gmid_op
    w_req = id_req / id_w_op
    power = VDD * id_req
    
    return {
        'gmid': gmid_op,
        'gain': gain_op,
        'ft': ft_op,
        'id': id_req,
        'w': w_req,
        'power': power
    }

design_A = get_design_point(8)
design_B = get_design_point(15)

print(f"{'Parameter':<20} {'Design A (gm/ID=8)':<25} {'Design B (gm/ID=15)':<25}")
print("=" * 70)
print(f"{'gm/ID (S/A)':<20} {design_A['gmid']:<25.2f} {design_B['gmid']:<25.2f}")
print(f"{'Gain (V/V)':<20} {design_A['gain']:<25.1f} {design_B['gain']:<25.1f}")
print(f"{'fT (GHz)':<20} {design_A['ft']/1e9:<25.1f} {design_B['ft']/1e9:<25.1f}")
print(f"{'ID (µA)':<20} {design_A['id']*1e6:<25.3f} {design_B['id']*1e6:<25.3f}")
print(f"{'W (µm)':<20} {design_A['w']*1e6:<25.3f} {design_B['w']*1e6:<25.3f}")
print(f"{'Power (µW)':<20} {design_A['power']*1e6:<25.2f} {design_B['power']*1e6:<25.2f}")
print("\n💡 Notice: Design B uses less power but has lower fT")

---
## Part 7: Export for xschem

Finally, we export the parameters for use in xschem.

In [ ]:
# Get final design values
L_final = L_dropdown.value
L_idx = np.argmin(np.abs(np.array(nmos.length) - L_final))
gmid_final = gmid_slider.value

valid = np.isfinite(gmid_data[L_idx, :]) & (gmid_data[L_idx, :] > 0)
valid_indices = np.where(valid)[0]
closest_idx = valid_indices[np.argmin(np.abs(gmid_data[L_idx, valid] - gmid_final))]

gmid_op = gmid_data[L_idx, closest_idx]
id_w_op = id_w_data[L_idx, closest_idx]
vgs_op = nmos.vgs[closest_idx]
id_final = gm_required / gmid_op
w_final = id_final / id_w_op

print("="*60)
print("PARAMETERS FOR XSCHEM")
print("="*60)
print(f"""
Symbol: sg13_lv_nmos (IHP SG13G2 LV NMOS)

Component properties:
  model = sg13_lv_nmos
  W = {w_final*1e6:.3f}u
  L = {L_final*1e6:.3f}u
  ng = 1
  m = 1

Expected operating point:
  VGS = {vgs_op:.3f} V
  VDS = {VDS_op:.2f} V
  ID  = {id_final*1e6:.3f} µA
  gm  = {gm_required*1e6:.3f} µS
""")

print("\n📋 Copy these values to xschem when creating your schematic.")
print("   The sg13_lv_nmos symbol is at:")
print("   $PDK_ROOT/ihp-sg13g2/libs.tech/xschem/sg13g2_pr/")

---
## Summary

In this tutorial you learned:

1. **gm/ID Methodology**: Use lookup tables instead of simplified equations
2. **The 4 fundamental charts**: fT, gain, ID/W, gm/W vs gm/ID
3. **Design process**:
   - Calculate required gm from BW and CL
   - Select L to meet gain requirement
   - Select gm/ID balancing trade-offs
   - Calculate W and ID
4. **Verification**: Use ngspice to validate the design
5. **Export**: Parameters ready for xschem

### Next Steps

- Create the schematic in xschem
- Simulate the complete amplifier (with load and current source)
- Analyze frequency response (.ac)
- Explore other blocks: differential pair, current mirror